In [10]:
!pip list

Package                   Version
------------------------- --------------
anyio                     4.10.0
appnope                   0.1.4
argon2-cffi               25.1.0
argon2-cffi-bindings      25.1.0
arrow                     1.3.0
asttokens                 3.0.0
async-lru                 2.0.5
attrs                     25.3.0
babel                     2.17.0
beautifulsoup4            4.13.5
bleach                    6.2.0
certifi                   2025.8.3
cffi                      2.0.0
charset-normalizer        3.4.3
comm                      0.2.3
debugpy                   1.8.16
decorator                 5.2.1
defusedxml                0.7.1
duckdb                    1.2.1
et_xmlfile                2.0.0
executing                 2.2.1
fastjsonschema            2.21.2
fqdn                      1.5.1
h11                       0.16.0
httpcore                  1.0.9
httpx                     0.28.1
idna                      3.10
ipykernel                 6.30.1
ipython         

In [ ]:
!pip install requests
!pip install pandas
!pip install plotly

Looking in indexes: https://pypi.python.org/simple, https://pypi.apple.com/simple

[notice] A new release of pip is available: 24.0 -> 25.2
[notice] To update, run: pip install --upgrade pip


In [3]:
import requests

API_KEY = 'QV8D-PY9K-9LMT-DWEI'

url = f"https://api.bart.gov/api/sched.aspx?cmd=arrive&orig=ASHB&dest=CIVC&date=now&key={API_KEY}&b=2&a=2&l=1&json=y"

print(url)

response = requests.get(url)

print(f"Status Code: {response.status_code}")

json_data = response.json()




https://api.bart.gov/api/sched.aspx?cmd=arrive&orig=ASHB&dest=CIVC&date=now&key=QV8D-PY9K-9LMT-DWEI&b=2&a=2&l=1&json=y
Status Code: 200


#json to pandas dataframe

In [4]:
import pandas as pd

df = pd.json_normalize(json_data)

print(df.shape)
df


(1, 12)


,?xml.@version,?xml.@encoding,root.@id,root.uri.#cdata-section,root.origin,root.destination,root.schedule.date,root.schedule.time,root.schedule.before,root.schedule.after,root.schedule.request.trip,root.message.legend
0,1.0,utf-8,1,http://api.bart.gov/api/sched.aspx?cmd=arrive&...,ASHB,CIVC,"Sep 14, 2025",10:19 AM,2,2,"[{'@origin': 'ASHB', '@destination': 'CIVC', '...",bikeflag: 1 = bikes allowed. 0 = no bikes allo...


In [5]:
import json

print(json.dumps(json_data, indent=2))

{
  "?xml": {
    "@version": "1.0",
    "@encoding": "utf-8"
  },
  "root": {
    "@id": "1",
    "uri": {
      "#cdata-section": "http://api.bart.gov/api/sched.aspx?cmd=arrive&orig=ASHB&dest=CIVC&date=now&b=2&a=2&l=1&json=y"
    },
    "origin": "ASHB",
    "destination": "CIVC",
    "schedule": {
      "date": "Sep 14, 2025",
      "time": "10:19 AM",
      "before": "2",
      "after": "2",
      "request": {
        "trip": [
          {
            "@origin": "ASHB",
            "@destination": "CIVC",
            "@fare": "4.80",
            "@origTimeMin": "09:38 AM",
            "@origTimeDate": "09/14/2025",
            "@destTimeMin": "10:04 AM",
            "@destTimeDate": "09/14/2025",
            "@clipper": "1.80",
            "@tripTime": "26",
            "fares": {
              "@level": "normal",
              "fare": [
                {
                  "@amount": "4.80",
                  "@class": "clipper",
                  "@name": "Clipper"
               

In [6]:
trips = json_data['root']['schedule']['request']['trip']

df_trips = pd.json_normalize(trips)
df_trips


,@origin,@destination,@fare,@origTimeMin,@origTimeDate,@destTimeMin,@destTimeDate,@clipper,@tripTime,leg,fares.@level,fares.fare
0,ASHB,CIVC,4.80,09:38 AM,09/14/2025,10:04 AM,09/14/2025,1.80,26,"[{'@order': '1', '@origin': 'ASHB', '@destinat...",normal,"[{'@amount': '4.80', '@class': 'clipper', '@na..."
1,ASHB,CIVC,4.80,09:46 AM,09/14/2025,10:11 AM,09/14/2025,1.80,25,"[{'@order': '1', '@origin': 'ASHB', '@destinat...",normal,"[{'@amount': '4.80', '@class': 'clipper', '@na..."
2,ASHB,CIVC,4.80,09:58 AM,09/14/2025,10:23 AM,09/14/2025,1.80,25,"[{'@order': '1', '@origin': 'ASHB', '@destinat...",normal,"[{'@amount': '4.80', '@class': 'clipper', '@na..."
3,ASHB,CIVC,4.80,10:06 AM,09/14/2025,10:31 AM,09/14/2025,1.80,25,"[{'@order': '1', '@origin': 'ASHB', '@destinat...",normal,"[{'@amount': '4.80', '@class': 'clipper', '@na..."


In [7]:
legs_df = pd.json_normalize(
            trips,
            record_path=['leg'],
)

legs_df


,@order,@origin,@destination,@origTimeMin,@origTimeDate,@destTimeMin,@destTimeDate,@originPlatform,@destinationPlatform,@line,@bikeflag,@trainHeadStation,@load
0,1,ASHB,MCAR,09:38 AM,09/14/2025,09:41 AM,09/14/2025,PL 2,PL 2,ROUTE 4,1,OAK Airport / Berryessa/North San Jose,0
1,2,MCAR,CIVC,09:42 AM,09/14/2025,10:04 AM,09/14/2025,PL 4,PL 1,ROUTE 1,1,San Francisco International Airport,0
2,1,ASHB,CIVC,09:46 AM,09/14/2025,10:11 AM,09/14/2025,PL 2,PL 1,ROUTE 7,1,SF / SFO Airport / Millbrae,0
3,1,ASHB,MCAR,09:58 AM,09/14/2025,10:01 AM,09/14/2025,PL 2,PL 2,ROUTE 4,1,OAK Airport / Berryessa/North San Jose,0
4,2,MCAR,CIVC,10:02 AM,09/14/2025,10:23 AM,09/14/2025,PL 4,PL 1,ROUTE 1,1,San Francisco International Airport,0
5,1,ASHB,CIVC,10:06 AM,09/14/2025,10:31 AM,09/14/2025,PL 2,PL 1,ROUTE 7,1,SF / SFO Airport / Millbrae,0


In [9]:
legs_df['@origTimeMin'] = pd.to_datetime(legs_df['@origTimeMin'])
legs_df['@destTimeMin'] = pd.to_datetime(legs_df['@destTimeMin'])

legs_df['duration'] = legs_df['@destTimeMin'] - legs_df['@origTimeMin']
legs_df['duration_min'] = legs_df['duration'].dt.total_seconds() / 60

legs_df

,@order,@origin,@destination,@origTimeMin,@origTimeDate,@destTimeMin,@destTimeDate,@originPlatform,@destinationPlatform,@line,@bikeflag,@trainHeadStation,@load,duration,duration_min
0,1,ASHB,MCAR,2025-09-14 09:38:00,09/14/2025,2025-09-14 09:41:00,09/14/2025,PL 2,PL 2,ROUTE 4,1,OAK Airport / Berryessa/North San Jose,0,0 days 00:03:00,3.0
1,2,MCAR,CIVC,2025-09-14 09:42:00,09/14/2025,2025-09-14 10:04:00,09/14/2025,PL 4,PL 1,ROUTE 1,1,San Francisco International Airport,0,0 days 00:22:00,22.0
2,1,ASHB,CIVC,2025-09-14 09:46:00,09/14/2025,2025-09-14 10:11:00,09/14/2025,PL 2,PL 1,ROUTE 7,1,SF / SFO Airport / Millbrae,0,0 days 00:25:00,25.0
3,1,ASHB,MCAR,2025-09-14 09:58:00,09/14/2025,2025-09-14 10:01:00,09/14/2025,PL 2,PL 2,ROUTE 4,1,OAK Airport / Berryessa/North San Jose,0,0 days 00:03:00,3.0
4,2,MCAR,CIVC,2025-09-14 10:02:00,09/14/2025,2025-09-14 10:23:00,09/14/2025,PL 4,PL 1,ROUTE 1,1,San Francisco International Airport,0,0 days 00:21:00,21.0
5,1,ASHB,CIVC,2025-09-14 10:06:00,09/14/2025,2025-09-14 10:31:00,09/14/2025,PL 2,PL 1,ROUTE 7,1,SF / SFO Airport / Millbrae,0,0 days 00:25:00,25.0


In [ ]:
trip_id = 0
trip_ids = []

for i, row in legs_df.iterrows():
    if i == 0:
        trip_id += 1
    else:
        prev = legs_df.iloc[i-1]
        # start new trip if current leg doesn't continue from previous leg
        if row['@origin'] != prev['@destination']:
            trip_id += 1
    trip_ids.append(trip_id)
    
legs_df['trip_id'] = trip_ids

legs_df

,@order,@origin,@destination,@origTimeMin,@origTimeDate,@destTimeMin,@destTimeDate,@originPlatform,@destinationPlatform,@line,@bikeflag,@trainHeadStation,@load,duration,duration_min,trip_id
0,1,ASHB,MCAR,2025-09-14 09:38:00,09/14/2025,2025-09-14 09:41:00,09/14/2025,PL 2,PL 2,ROUTE 4,1,OAK Airport / Berryessa/North San Jose,0,0 days 00:03:00,3.0,1
1,2,MCAR,CIVC,2025-09-14 09:42:00,09/14/2025,2025-09-14 10:04:00,09/14/2025,PL 4,PL 1,ROUTE 1,1,San Francisco International Airport,0,0 days 00:22:00,22.0,1
2,1,ASHB,CIVC,2025-09-14 09:46:00,09/14/2025,2025-09-14 10:11:00,09/14/2025,PL 2,PL 1,ROUTE 7,1,SF / SFO Airport / Millbrae,0,0 days 00:25:00,25.0,2
3,1,ASHB,MCAR,2025-09-14 09:58:00,09/14/2025,2025-09-14 10:01:00,09/14/2025,PL 2,PL 2,ROUTE 4,1,OAK Airport / Berryessa/North San Jose,0,0 days 00:03:00,3.0,3
4,2,MCAR,CIVC,2025-09-14 10:02:00,09/14/2025,2025-09-14 10:23:00,09/14/2025,PL 4,PL 1,ROUTE 1,1,San Francisco International Airport,0,0 days 00:21:00,21.0,3
5,1,ASHB,CIVC,2025-09-14 10:06:00,09/14/2025,2025-09-14 10:31:00,09/14/2025,PL 2,PL 1,ROUTE 7,1,SF / SFO Airport / Millbrae,0,0 days 00:25:00,25.0,4


In [ ]:
routes_df = (
    legs_df.groupby("trip_id")
    .agg(
        route_origin=("@origin", "first"),
        route_destination=("@destination", "last"),
        total_duration=("duration_min", "sum")
        )
    .reset_index()
)

legs_df = legs_df.merge(routes_df, on="trip_id", how="left")

legs_df

,@order,@origin,@destination,@origTimeMin,@origTimeDate,@destTimeMin,@destTimeDate,@originPlatform,@destinationPlatform,@line,@bikeflag,@trainHeadStation,@load,duration,duration_min,trip_id,route_origin,route_destination,total_duration
0,1,ASHB,MCAR,2025-09-14 09:38:00,09/14/2025,2025-09-14 09:41:00,09/14/2025,PL 2,PL 2,ROUTE 4,1,OAK Airport / Berryessa/North San Jose,0,0 days 00:03:00,3.0,1,ASHB,CIVC,25.0
1,2,MCAR,CIVC,2025-09-14 09:42:00,09/14/2025,2025-09-14 10:04:00,09/14/2025,PL 4,PL 1,ROUTE 1,1,San Francisco International Airport,0,0 days 00:22:00,22.0,1,ASHB,CIVC,25.0
2,1,ASHB,CIVC,2025-09-14 09:46:00,09/14/2025,2025-09-14 10:11:00,09/14/2025,PL 2,PL 1,ROUTE 7,1,SF / SFO Airport / Millbrae,0,0 days 00:25:00,25.0,2,ASHB,CIVC,25.0
3,1,ASHB,MCAR,2025-09-14 09:58:00,09/14/2025,2025-09-14 10:01:00,09/14/2025,PL 2,PL 2,ROUTE 4,1,OAK Airport / Berryessa/North San Jose,0,0 days 00:03:00,3.0,3,ASHB,CIVC,24.0
4,2,MCAR,CIVC,2025-09-14 10:02:00,09/14/2025,2025-09-14 10:23:00,09/14/2025,PL 4,PL 1,ROUTE 1,1,San Francisco International Airport,0,0 days 00:21:00,21.0,3,ASHB,CIVC,24.0
5,1,ASHB,CIVC,2025-09-14 10:06:00,09/14/2025,2025-09-14 10:31:00,09/14/2025,PL 2,PL 1,ROUTE 7,1,SF / SFO Airport / Millbrae,0,0 days 00:25:00,25.0,4,ASHB,CIVC,25.0


In [ ]:
import plotly.express as px

# timeline
fig = px.timeline(
    legs_df,
    x_start='@origTimeMin',
    x_end='@destTimeMin',
    y='trip_id',
    color='@line',
    hover_data=['@origin', '@destination', 'duration_min']
)

# bar thickness
fig.update_traces(width=0.3)

for i, row in legs_df.iterrows():
    # origin label
    fig.add_annotation(
        x=row['@origTimeMin'],
        y=row['trip_id'],
        text=row['@origin'],
        showarrow=False,
        xanchor='center',
        yanchor='middle',
        textangle=-90,
        font=dict(size=10, color="black")
    )
    # destination label
    fig.add_annotation(
        x=row['@destTimeMin'],
        y=row['trip_id'],
        text=row['@destination'],
        showarrow=False,
        xanchor='center',
        yanchor='middle',
        textangle=-90,
        font=dict(size=10, color="black")
    )

# total duration label
for trip_id, total_duration in legs_df.groupby('trip_id')['total_duration'].first().items():
    
    end_time = legs_df.loc[legs_df['trip_id'] == trip_id, '@destTimeMin'].max()
    
    fig.add_annotation(
        x=end_time + pd.Timedelta(minutes=2),
        y=trip_id + 0.35,
        text=f"Total Duration: {total_duration:.0f} min",
        showarrow=False,
        font=dict(size=10, color="black"),
        align="left"
    )

# gridlines
fig.update_xaxes(
    title='Time',
    showgrid=True,
    dtick=1*60*1000
)
fig.update_yaxes(title='Trip ID', showgrid=True)

fig.show()
